# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/). 

# Part One: Building a company list

Search for businesses with the word **WRECK** in their names.

* **Tip:** Start by scraping the first page to a dataframe, then expand to a loop that combines all of the pages. Finally combine all of the dataframes with `pd.concat`. You might find [this page helpful](https://jonathansoma.com/everything/scraping/pagination/), although the "On an interactive site" example uses Selenium instead of Playwright. You should be able to figure out how to change it!
* **Tip:** You can't just do a `try`/`except`, because even if you ask for page 99999 it will always give you the last page again! Watch out that you don't get stuck in an infinite loop!

In [1]:
from playwright.async_api import async_playwright
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()
await page.goto("https://www.tdlr.texas.gov/tools_search/")

<Response url='https://www.tdlr.texas.gov/tools_search/' request=<Request url='https://www.tdlr.texas.gov/tools_search/' method='GET'>>

In [2]:
await page.locator("#namebutton").click()

In [3]:
await page.locator("#namedata").fill("WRECK")

In [7]:
await page.get_by_text("Search", exact=True).click()

In [8]:
import pandas as pd
html = await page.content()
tables = pd.read_html(html)

/var/folders/21/d_7v4w8j3jg95zjx8lfqpw800000gn/T/ipykernel_75609/958859366.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


In [29]:
df = tables[2]

In [30]:
headers = df.iloc[0]
headers

0       Customer
1       DBA Name
2    TDLR Number
3           City
4          State
5       Zip code
Name: 0, dtype: object

In [31]:
df = pd.DataFrame(df.values[1:], columns=headers)

In [32]:
df

,Customer,DBA Name,TDLR Number,City,State,Zip code
0,WRIGHT'S WRECKER SERVICE,NaN,0593304VSF (Expired!),NASH,TX,75569
1,WRIGHT'S WRECKER SERVICE,NaN,0508198VSF,NASH,TX,75569
2,XTREME WRECKER SERVICES INC.,NaN,0650507VSF,BARSTOW,TX,79719
3,"XTREME WRECKER SERVICES, INC",NaN,006504760C,BARSTOW,TX,79719


In [14]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [33]:
all_tables = []
for i in range(41):
    url = f"https://www.tdlr.texas.gov/tools_search/mccs_search_process.asp?page={i+1}&stype=name&ls_temp=WRECK&ls_carrier_type=COMPANY"
    await page.goto(url)
    await page.get_by_text("DBA Name").wait_for()
    html = await page.content()
    tables = pd.read_html(html)
    df = tables[2]
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)
    all_tables.append(df)

In [34]:
all_tables[40]

,Customer,DBA Name,TDLR Number,City,State,Zip code
0,WRIGHT'S WRECKER SERVICE,NaN,0593304VSF (Expired!),NASH,TX,75569
1,WRIGHT'S WRECKER SERVICE,NaN,0508198VSF,NASH,TX,75569
2,XTREME WRECKER SERVICES INC.,NaN,0650507VSF,BARSTOW,TX,79719
3,"XTREME WRECKER SERVICES, INC",NaN,006504760C,BARSTOW,TX,79719


In [38]:
list_df = pd.concat(all_tables).reset_index(drop=True)
list_df

,Customer,DBA Name,TDLR Number,City,State,Zip code
0,CALIBER WRECKER SERVICE LLC,NaN,006598046C (Insurance not applied !),CHANNELVIEW,TX,77530
1,1ST CHOICE WRECKER SERVICE,"1ST CHOICE PAINT & BODY, INC",006096604C,TERRELL,TX,75160
2,1ST CHOICE WRECKER SERVICE,"1ST CHOICE PAINT & BODY, INC.",0612137VSF,TERRELL,TX,75160
3,1ST CHOICE WRECKER SERVICE LLC,NaN,006529369C,SILSBEE,TX,77656
4,1ST CHOICE WRECKER SERVICE LLC,NaN,0652937VSF,SILSBEE,TX,77656
...,...,...,...,...,...,...
799,WRIGHT'S WRECKER SERVICE,NaN,005388553C,NASH,TX,75569
800,WRIGHT'S WRECKER SERVICE,NaN,0593304VSF (Expired!),NASH,TX,75569
801,WRIGHT'S WRECKER SERVICE,NaN,0508198VSF,NASH,TX,75569
802,XTREME WRECKER SERVICES INC.,NaN,0650507VSF,BARSTOW,TX,79719


### Cleanup

If you haven't already, rename the columns to be:
    
    * Customer
    * DBA Name
    * TDLR Number
    * City
    * State
    * Zip code

and remove all of the rows where the customer name is `Customer`.

In [39]:
list_df[list_df['Customer'] != 'Customer']

,Customer,DBA Name,TDLR Number,City,State,Zip code
0,CALIBER WRECKER SERVICE LLC,NaN,006598046C (Insurance not applied !),CHANNELVIEW,TX,77530
1,1ST CHOICE WRECKER SERVICE,"1ST CHOICE PAINT & BODY, INC",006096604C,TERRELL,TX,75160
2,1ST CHOICE WRECKER SERVICE,"1ST CHOICE PAINT & BODY, INC.",0612137VSF,TERRELL,TX,75160
3,1ST CHOICE WRECKER SERVICE LLC,NaN,006529369C,SILSBEE,TX,77656
4,1ST CHOICE WRECKER SERVICE LLC,NaN,0652937VSF,SILSBEE,TX,77656
...,...,...,...,...,...,...
799,WRIGHT'S WRECKER SERVICE,NaN,005388553C,NASH,TX,75569
800,WRIGHT'S WRECKER SERVICE,NaN,0593304VSF (Expired!),NASH,TX,75569
801,WRIGHT'S WRECKER SERVICE,NaN,0508198VSF,NASH,TX,75569
802,XTREME WRECKER SERVICES INC.,NaN,0650507VSF,BARSTOW,TX,79719


## Save as `wreckers.csv`

In [41]:
list_df.to_csv("wreckers.csv", index=False)

# Part Two: Company info

> You can use whatever tool you'd like for this, but form submission doesn't necessarily mean Playwright! If you want to go the `requests` route instead, feel free to [look at this page](https://jonathansoma.com/everything/scraping/pretending-to-be-a-browser/) about several ways to pretend to be a browser.

## Step 1: Scraping one page

Try searching from the [tools page](https://www.tdlr.texas.gov/tools_search/) for the TDLR Number `006556161C`. From the results page, scrape the:

* Business name
* Phone number
* License status
* Physical address

And save the results into a dictionary. Also include `TDLR Number` in the dictionary. **Print the dictionary**.

> ***Tip:** It's best if each item has its own key, but **it's fine to pull "larger" sections of the page and split them up in pandas later on***
>
> ***Tip:** Be sure you don't forget to include `TDLR Number` in the dictionary!*

In [42]:
await page.goto("https://www.tdlr.texas.gov/tools_search/")
await page.locator("#mcrbutton").click()
await page.locator("#mcrdata").fill("006556161C")

In [43]:
await page.get_by_text("Search", exact=True).click()

In [47]:
from bs4 import BeautifulSoup
html = await page.content()
doc = BeautifulSoup(html)

In [91]:
table1 = doc.find('table', align='center', border="0", cellpadding="0", cellspacing="0", width="")
info = table1.find_all('td')
Business_name = info[2].text.split('\xa0')[-1]
Phone_number = info[-2].text.split('\xa0')[-1]

In [86]:
table2 = doc.find('table', align='center', border="1", cellpadding="1", cellspacing="0")
info = table2.find_all('td')
License_status = info[1].text.split('\xa0')[-1]
Physical_address = info[-1].text.split('Physical:')[-1].replace('\n', '').replace('\t', '').replace('\xa0', '')

In [92]:
dic = {
    'TDLR_Number': '006556161C',
    'Business_name': Business_name,
    'Phone_number': Phone_number,
    'License_status': License_status,
    'Physical_address': Physical_address.strip()
}
dic

{'TDLR_Number': '006556161C',
 'Business_name': 'DIRTY SOUTH TRANSPORT AND RECOVERY, LLC ',
 'Phone_number': '713-259-5445',
 'License_status': 'Active',
 'Physical_address': '11053 LORETTA LN    PLANTERSVILLE,TX.77363'}

## Step 2: Move into one cell

Move the code above all into one cell that relies on the variable `tdlr_number`.

Add the code below to the page and confirm that it displays the data for the correct result.

```python
tdlr_number = '0654479VSF'
```

Confirm that the information is correct. Did it not work out? Go back and edit your selectors, or be a little broader in the parts of the page you sweep up.

In [97]:
tdlr_number = '0654479VSF'
await page.goto("https://www.tdlr.texas.gov/tools_search/")
await page.locator("#mcrbutton").click()
await page.locator("#mcrdata").fill(tdlr_number)
await page.get_by_text("Search", exact=True).click()
html = await page.content()
doc = BeautifulSoup(html)
table1 = doc.find('table', align='center', border="0", cellpadding="0", cellspacing="0", width="")
info = table1.find_all('td')
Business_name = info[2].text.split('\xa0')[-1]
Phone_number = info[-2].text.split('\xa0')[-1]
table2 = doc.find('table', align='center', border="1", cellpadding="1", cellspacing="0")
info = table2.find_all('td')
License_status = info[1].text.split('\xa0')[-1]
Physical_address = info[-1].text.split('Physical:')[-1].replace('\n', '').replace('\t', '').replace('\xa0', '')
dic = {
    'TDLR_Number': tdlr_number,
    'Business_name': Business_name,
    'Phone_number': Phone_number,
    'License_status': License_status,
    'Physical_address': Physical_address.strip()
}
dic

{'TDLR_Number': '0654479VSF',
 'Business_name': '24/7TOWINGANDRECOVERYLLC',
 'Phone_number': '4325576733',
 'License_status': 'Active',
 'Physical_address': '3601 N COUNTY ROAD 1148    MIDLAND,TX.79705'}

## Step 3: Scraping many pages

Using pandas, read in `trucks-subset.csv`.

In [98]:
import pandas as pd
df = pd.read_csv('trucks-subset.csv')
df.head()

,TDLR Number
0,006565540C
1,0654479VSF
2,006564940C


## Scrape every single row, creating a list of dictionaries from the scraped data.

You should never use for loops with pandas *except when working with Playwright*. To use loops in pandas you'll make use of `.iterrows()`.

The code below loops through a dataframe called `df` and prints out the `address` column.

```python
for index, row in df.iterrows():
    print(row['address'])
```

You'll adapt this code to use your dataframe, and combine it with the scraping code you wrote above.

> ***Tip:** This is like what we did for the townships in class*

In [99]:
for index, row in df.iterrows():
    print(row['TDLR Number'])

006565540C
0654479VSF
006564940C


In [100]:
all_dic = []
for index, row in df.iterrows():
    tdlr_number = row['TDLR Number']
    await page.goto("https://www.tdlr.texas.gov/tools_search/")
    await page.locator("#mcrbutton").click()
    await page.locator("#mcrdata").fill(tdlr_number)
    await page.get_by_text("Search", exact=True).click()
    html = await page.content()
    doc = BeautifulSoup(html)
    table1 = doc.find('table', align='center', border="0", cellpadding="0", cellspacing="0", width="")
    info = table1.find_all('td')
    Business_name = info[2].text.split('\xa0')[-1]
    Phone_number = info[-2].text.split('\xa0')[-1]
    table2 = doc.find('table', align='center', border="1", cellpadding="1", cellspacing="0")
    info = table2.find_all('td')
    License_status = info[1].text.split('\xa0')[-1]
    Physical_address = info[-1].text.split('Physical:')[-1].replace('\n', '').replace('\t', '').replace('\xa0', '')
    dic = {
    'TDLR_Number': tdlr_number,
    'Business_name': Business_name,
    'Phone_number': Phone_number,
    'License_status': License_status,
    'Physical_address': Physical_address.strip()
    }
    all_dic.append(dic)
all_dic

[{'TDLR_Number': '006565540C',
  'Business_name': 'H & A TOWING LLC ',
  'Phone_number': '512-999-8883',
  'License_status': 'Expired',
  'Physical_address': '11710 JOSEPH CLAYTON DR    AUSTIN,TX.78753'},
 {'TDLR_Number': '0654479VSF',
  'Business_name': '24/7TOWINGANDRECOVERYLLC',
  'Phone_number': '4325576733',
  'License_status': 'Active',
  'Physical_address': '3601 N COUNTY ROAD 1148    MIDLAND,TX.79705'},
 {'TDLR_Number': '006564940C',
  'Business_name': 'A&NTOWINGLLC',
  'Phone_number': '2106678546',
  'License_status': 'Active',
  'Physical_address': '1341  GAYLE LN    POTEET,TX.78065'}]

## Save your dataframe as `data-uncleaned.csv`

In [101]:
TDLR_df = pd.DataFrame(all_dic)
TDLR_df.to_csv("data-uncleaned.csv", index=False)
TDLR_df.head()

,TDLR_Number,Business_name,Phone_number,License_status,Physical_address
0,006565540C,H & A TOWING LLC,512-999-8883,Expired,"11710 JOSEPH CLAYTON DR AUSTIN,TX.78753"
1,0654479VSF,24/7TOWINGANDRECOVERYLLC,4325576733,Active,"3601 N COUNTY ROAD 1148 MIDLAND,TX.79705"
2,006564940C,A&NTOWINGLLC,2106678546,Active,"1341 GAYLE LN POTEET,TX.78065"


# Cleaning your data

## Re-open the `data-uncleaned.csv` file

You probably want to set `pd.options.display.max_colwidth`

In [102]:
df = pd.read_csv('data-uncleaned.csv')
df.head()

,TDLR_Number,Business_name,Phone_number,License_status,Physical_address
0,006565540C,H & A TOWING LLC,512-999-8883,Expired,"11710 JOSEPH CLAYTON DR AUSTIN,TX.78753"
1,0654479VSF,24/7TOWINGANDRECOVERYLLC,4325576733,Active,"3601 N COUNTY ROAD 1148 MIDLAND,TX.79705"
2,006564940C,A&NTOWINGLLC,2106678546,Active,"1341 GAYLE LN POTEET,TX.78065"


## Clean it up!

Make sure there are columns for

- Business name
- Phone number
- License status
- Physical address

And drop all of the other columns (The easiest way is to use `df = df.drop(columns=[...])`)

## Combine with the original CSV file

To combine dataframes based on indexes, you use `df.join(other_df)`. If you'd prefer to match by columns, you could also use `df.merge(other_df, left_on='...', right_on='...')` and tell it the two columns that match between the two.

### Save to a CSV